<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/BlueSky_RM/BlueSky_Airlines_Network_Revenue_Management_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a template for the **BlueSky Airlines Network Revenue Management** mini-case. 

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [ ]:
#@markdown We first import some useful modules. 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

from IPython.display import display

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# some modules to create local directories for CBC (to avoid issues with solving multiple models)
import os
def new_local_directory(name):
    full_path = os.path.join(".", name)
    os.makedirs(full_path, exist_ok=True)
    return full_path

# install the latest version of seaborn for nicer graphics
#!pip install --prefix {sys.prefix} seaborn==0.11.0  &> /dev/null
import seaborn as sns

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [ ]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://github.com/dan-a-iancu/airm/blob/master/BlueSky_RM/BlueSky_Data.xlsx?raw=true'
local_file = "BlueSky_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('BlueSky_Data.xlsx', <http.client.HTTPMessage at 0x7f310e991050>)

Read in and store the data in suitable dataframes.

In [ ]:
#@markdown Create dataframes based on the Excel file

# Read in all the information about the itinerarys sold, and the aircraft capacities
itineraryData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Itinerary_data", index_col=0)
display(itineraryData)

aircraftData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Aircraft_data", index_col=0)
display(aircraftData)

,Q_Fare,Y_Fare,Q_Demand,Y_Demand,BOS_ORD_Flight,JFK_ORD_Flight,ORD_SFO_Flight,ORD_LAX_Flight
BOS_ORD,200,220,25,20,1,0,0,0
BOS_SFO,320,420,55,40,1,0,1,0
BOS_LAX,400,490,65,25,1,0,0,1
JFK_ORD,250,290,24,16,0,1,0,0
JFK_SFO,410,540,65,50,0,1,1,0
JFK_LAX,450,550,40,35,0,1,0,1
ORD_SFO,210,230,21,50,0,0,1,0
ORD_LAX,260,300,25,14,0,0,0,1


,Capacity
BOS_ORD_Flight,200
JFK_ORD_Flight,200
ORD_SFO_Flight,200
ORD_LAX_Flight,200


## Create Python lists based on the data-frames

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [ ]:
#@markdown Create useful lists
numODPairs = len(itineraryData)               # number of origin-destination (O-D) pairs used for itinerarys
allODPairs = list(itineraryData.index)        # a list with all the O-D pairs (rows in itineraryData)
numFlights = len(aircraftData)             # number of distinct flights/legs operated by the Airline
allFlights = list(aircraftData.index)      # a list with all the flights operated (rows in aircraftData)

# have a look
print(allODPairs)
print(allFlights)

['BOS_ORD', 'BOS_SFO', 'BOS_LAX', 'JFK_ORD', 'JFK_SFO', 'JFK_LAX', 'ORD_SFO', 'ORD_LAX']
['BOS_ORD_Flight', 'JFK_ORD_Flight', 'ORD_SFO_Flight', 'ORD_LAX_Flight']


<font color=red>**IMPORTANT HINT.**</font> When defining your **decision variables** in this model, you may want to consider *matrices* (or grids) of decision variables instead of a simple list like we've created before. Specifically, you may want to add a decision variable for every pair of elements where the first element is from one list and the second element is from another list. 

When the decision variables are organized in a grid, it's significantly easier to use the ``addVars`` function to add all of the decision variables to the model at once. The function ``addVars`` can take as arguments several lists, e.g., `addVars(list1, list2, list3,...)`, in which case it will return one decision variable for every possible tuple of values. These decision variables will be stored as a dictionary, with the dictionary keys corresponding to the tuple of values. 

For instance, suppose `list1 = ["Paris", "Budapest", "Beijing"]` and `list2= [1,2]`. Then `myDecisions = addVars(list1,list2)` would return a dictionary containing one decision for every pair, i.e., ("Paris",1), ("Paris",2), ("Budapest",1), ("Budapest",2), etc. To access the decision corresponding to the pair ("Paris",1), we can simply use `myDecisions[("Paris",1)]` or `myDecisions["Paris",1]`.

_____
# **Q1**